Connected to Python 3.10.11

### Load libraries

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

# Directly import helper function
notebook_dir = os.getcwd()
src_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\src"
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import add_src_to_path
add_src_to_path()

from data_split import split_time_series_by_river
from rf_model import train_and_apply_rf_with_tuning
from plot_predictions import plot_predictions_by_river
from plot_predictions import plot_actual_vs_predicted

# Choose from "Bristol Bay", "Fraser River" and "Columbia River"
river_system = "Fraser River"

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # assumes notebook is in /notebooks
data_path = os.path.join(project_root, 'data', 'Combined_FeatureSet_For_Model.csv')

combined_df = pd.read_csv(data_path)
# Optional: Select river system
combined_df = combined_df[combined_df["System"] == river_system]
# Optional: Select river
#combined_df = combined_df[combined_df["River"] == "Alagnak"]

combined_df.columns

if True:
    features_to_lag = ['Total_Returns', 'AgeClass_0.1',
       'AgeClass_0.2', 'AgeClass_0.3', 'AgeClass_0.4', 'AgeClass_0.5',
       'AgeClass_1.1', 'AgeClass_1.2', 'AgeClass_1.3', 'AgeClass_1.4',
       'AgeClass_1.5', 'AgeClass_2.1', 'AgeClass_2.2', 'AgeClass_2.3',
       'AgeClass_2.4', 'AgeClass_3.1', 'AgeClass_3.2', 'AgeClass_3.3',
       'AgeClass_3.4', 'Total_Returns_NextYear', 'Pacea_ALPI_Anomaly',
       'npi_mean_NovMar', 'oni_mean_DecFeb', 'npgo_mean_DecFeb',
       'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep']
    for feat in features_to_lag:
        for lag in [1, 2, 3, 4, 5]:
            combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)

            # Optional: Standardize Total_Returns_NextYear
if (False):
    # Step 1: Compute per-river return stats
    return_stats = combined_df.groupby('River')['Total_Returns_NextYear'].agg(
        returns_mean='mean',
        returns_std='std'
    ).reset_index()

    # Step 2: Merge stats temporarily for scaling
    combined_df = combined_df.merge(return_stats, on='River', how='left')

    # Step 3: Standardize Total_Returns_NextYear
    combined_df['Total_Returns_NextYear'] = (
        (combined_df['Total_Returns_NextYear'] - combined_df['returns_mean']) /
        combined_df['returns_std']
    )

    # Step 4: Drop the extra columns again
    combined_df = combined_df.drop(columns=['returns_mean', 'returns_std'])

    # Optional: Keep Spawner data and remove river Ugashik and first four year (1963-1966) 
# as no data available 
if False:
    combined_df = combined_df.dropna(subset=['total_spawners_y_minus_2_to_4'])
    combined_df = combined_df.dropna(subset=['AgeClass_0.2_Yminus5'])

missing_summary = combined_df.isnull().sum()
missing_cols = missing_summary[missing_summary > 0]


combined_df = combined_df.drop(columns=missing_cols.index)

train_df, test_df = split_time_series_by_river(
    combined_df,
    time_column="Year",
    group_columns=["System", "River"],
    test_fraction=0.2,
    gap_years=0  # Set to 1 if you want a 1-year gap between train and test
)

train_df["River_Name"] = train_df["River"] # For visualization
test_df["River_Name"] = test_df["River"]

train_df_encoded = pd.get_dummies(train_df, columns=["River"], prefix="River")
test_df_encoded = pd.get_dummies(test_df, columns=["River"], prefix="River")

model_list = ["RF", "GBRT", "XGB", "LR", "PR"]
all_results = {}

for model_name in model_list:
    print(f"\n===================== {model_name} =====================")
    try:
        results = train_and_apply_rf_with_tuning(
            model=model_name,
            train_df=train_df_encoded,
            test_df=test_df_encoded,
            topk_feat=10
        )

        all_results[model_name] = results  # Save results here

        print(f"✅ R2 Train: {results['R2_train']:.4f}")
        print(f"✅ R2 Test : {results['R2']:.4f}")
        print(f"📉 MSE     : {results['MSE']:.2f}")
        print(f"📊 MAPE    : {results['MAPE']:.2f}%")
        if results['Best_Params'] is not None:
            print(f"🔧 Best Params: {results['Best_Params']}")
        else:
            print("ℹ️ No parameter tuning applied.")

        # ✅ Print per river
        print("\n📍 Metrics by River (Test):")
        print(results['Metrics_by_River_Test'].round(2).to_string(index=False))

        print("\n📍 Metrics by River (Train):")
        print(results['Metrics_by_River_Train'].round(2).to_string(index=False))

    except Exception as e:
        print(f"❌ Error while running model {model_name}: {e}")



merged_metrics = []

for model_name, result in all_results.items():
    test_df = result['Metrics_by_River_Test'].copy()
    train_df = result['Metrics_by_River_Train'].copy()

    # Rename metric columns
    test_df = test_df.rename(columns={"R2": "R2_Test", "MSE": "MSE_Test", "MAPE": "MAPE_Test"})
    train_df = train_df.rename(columns={"R2": "R2_Train", "MSE": "MSE_Train", "MAPE": "MAPE_Train"})

    # Merge test and train on River_Name
    merged_df = pd.merge(test_df, train_df, on="River_Name", how="outer")

    # Add model name
    merged_df.insert(0, "Model", model_name)

    # Optional: Add System info (lookup from train_df or test_df)
    # You only need to do this once, so use one of the full dataframes
    river_system_map = pd.concat([train_df, test_df], ignore_index=True)[["River_Name"]].drop_duplicates()
    river_system_lookup = pd.concat([train_df_encoded, test_df_encoded], ignore_index=True)[["River_Name", "System"]].drop_duplicates()

    merged_df = pd.merge(merged_df, river_system_lookup, on="River_Name", how="left")

    merged_metrics.append(merged_df)


    final_df = pd.concat(merged_metrics, ignore_index=True)

    output_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\results.csv"
    final_df.to_csv(output_path, index=False)

    print(f"\n✅ Final metrics with system saved to:\n{output_path}")



C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_18832\1815902991.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_18832\1815902991.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_18832\1815902991.py:45: PerformanceWarning: DataFra


===================== RF =====================
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_2.2', 'AgeClass_2.3', 'pdo_mean_MaySep', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
RF R2: 0.49
RF MSE: 316008924972.28
RF MAPE: 558.93
✅ R2 Train: 0.8347
✅ R2 Test : 0.4870
📉 MSE     : 316008924972.28
📊 MAPE    : 558.93%
🔧 Best Params: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

📍 Metrics by River (Test):
 River_Name     R2          MSE    MAPE
     Chilko   0.26 9.990875e+11  123.50
Late Stuart -18.08 3.520742e+11  888.25
    Quesnel   0.67 2.122437e+11 1425.84
       Raft  -3.10 6.328142e+08  264.00
   Stellako   0.88 1.600643e+10   93.07

📍 Metrics by River (Train):
 River_Name     R2          MSE   MAPE
     Chilko   0.79 2.381062e+11  40.51
Late Stua

c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:145: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 1}
GBRT R2: 0.23
GBRT MSE: 472923345222.29
GBRT MAPE: 682.68
✅ R2 Train: 0.9975
✅ R2 Test : 0.2323
📉 MSE     : 472923345222.29
📊 MAPE    : 682.68%
🔧 Best Params: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 1}

📍 Metrics by River (Test):
 River_Name     R2          MSE    MAPE
     Chilko   0.20 1.085816e+12  133.90
Late Stuart -42.32 7.993017e+11  718.74
    Quesnel   0.30 4.524269e+11 1806.26
       Raft -63.75 1.000281e+10  678.59
   Stellako   0.87 1.706957e+10   75.90

📍 Metrics by River (Train):
 River_Name    R2          MSE   MAPE
     Chilko  1.00 4.511670e+09   7.61
Late Stuart  0.99 1.010279e+10 367.11
    Quesnel  1.00 2.859021e+09 596.54
       Raft -0.17 8.677259e+08 152.99
   Stellako  0.93 7.394838e+09  23.02

===================== XGB =====================
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'AgeClass_1.2',

c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:145: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100}
XGB R2: 0.12
XGB MSE: 541215021291.21
XGB MAPE: 608.66
✅ R2 Train: 0.9840
✅ R2 Test : 0.1215
📉 MSE     : 541215021291.21
📊 MAPE    : 608.66%
🔧 Best Params: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100}

📍 Metrics by River (Test):
 River_Name     R2          MSE    MAPE
     Chilko   0.41 8.006628e+11  129.30
Late Stuart -75.49 1.411350e+12  913.31
    Quesnel   0.28 4.687033e+11 1311.43
       Raft -13.65 2.263041e+09  583.76
   Stellako   0.82 2.309588e+10  105.48

📍 Metrics by River (Train):
 River_Name    R2          MSE    MAPE
     Chilko  0.98 1.991929e+10   17.87
Late Stuart  0.92 6.919664e+10  589.35
    Quesnel  0.99 4.357125e+10 1545.71
       Raft -1.11 1.560676e+09  288.36
   Stellako  0.70 3.082048e+10   44.26

===================== LR =====================
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_2.2', 'AgeClass_2.

c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:145: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

### Optional: Fit ARIMA model on residuals (only works if only one river selected so far)

In [ ]:
if True:    
    residuals = results["Timeline_train"]["Actual"] - results["Timeline_train"]["Predicted"]

    from statsmodels.tsa.arima.model import ARIMA
    residuals_series = pd.Series(residuals.values, index=results["Timeline_train"]["Year"])
    arima_model = ARIMA(residuals_series, order=(1,0,0))  # You may want to auto-tune this
    arima_fit = arima_model.fit()
    residual_forecast = arima_fit.forecast(steps=len(results["Timeline_test"]["Predicted"]))

    hybrid_pred = results["Timeline_test"]["Predicted"] + residual_forecast.values
    from sklearn.metrics import mean_squared_error, r2_score
    r2 = r2_score(results["Timeline_test"]["Actual"], hybrid_pred)
    print(r2)

    results["Timeline_test"]["Predicted"] = hybrid_pred

### Performance metrics

In [ ]:
results["Metrics_by_System"]

In [ ]:
results["Metrics_by_River"]

### Plot predictions

In [ ]:
plot_predictions_by_river(results["Timeline_test"])

In [ ]:
plot_predictions_by_river(results["Timeline_train"])

In [ ]:
# Plot Predicted vs Actual
plot_actual_vs_predicted(results)

### Feature Importances

In [ ]:
sorted_items = sorted(results["Feature_Importances"].items(), key=lambda x: x[1], reverse=True)
features, importances = zip(*sorted_items)

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(features, importances, color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importances')
plt.gca().invert_yaxis()  # Most important on top
plt.tight_layout()
plt.show()